In [1]:
import sqlalchemy as db
import pandas as pd

In [2]:
# Establish connection with engine

engine = db.create_engine('sqlite:///sqlite_db_pythonsqlite.db')
con = engine.connect()

In [3]:
metadata = db.MetaData()
members = db.Table('Members', metadata, autoload_with=engine)

f_metadata = db.MetaData()
facilities = db.Table('Facilities', f_metadata, autoload_with=engine)

b_metadata = db.MetaData()
bookings = db.Table('Bookings', b_metadata, autoload_with=engine)

/tmp/ipykernel_338755/3551606087.py:2: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['2']; using no arguments.
  members = db.Table('Members', metadata, autoload_with=engine)
/tmp/ipykernel_338755/3551606087.py:2: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['5']; using no arguments.
  members = db.Table('Members', metadata, autoload_with=engine)
/tmp/ipykernel_338755/3551606087.py:5: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['1']; using no arguments.
  facilities = db.Table('Facilities', f_metadata, autoload_with=engine)
/tmp/ipykernel_338755/3551606087.py:5: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['5']; using no arguments.
  facilities = db.Table('Facilities', f_metadata, autoload_with=engine)
/tmp/ipykernel_338755/3551606087.py:5: SAWar

# Q10

In [4]:
query = """SELECT
	f.name,
	guest_revenue + member_revenue AS total_revenue
FROM
	(SELECT
		facid,
		SUM(slots) * guestcost AS guest_revenue
	 FROM Bookings
     LEFT JOIN Facilities
     USING(facid)
	 WHERE memid = 0
	 GROUP BY facid) AS guest
LEFT JOIN
	(SELECT
   		facid,
   		SUM(slots) * membercost AS member_revenue
	 FROM Bookings
     LEFT JOIN Facilities
     USING(facid)
	 WHERE memid <> 0
	 GROUP BY facid) AS member
USING(facid)
LEFT JOIN Facilities AS f
USING(facid)
WHERE guest_revenue + member_revenue < 1000
ORDER BY total_revenue;"""

In [5]:
revenue_rs = con.execute(db.text(query))
revenue_df = pd.DataFrame(revenue_rs.fetchall())
revenue_df

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


# Q11

In [6]:
recommenders = members.alias()
stmt = db.select(
        members.c.surname.label('Member_Surname'),
        members.c.firstname.label('Member_Firstname'),
        recommenders.c.surname.label('Recommender_Surname'),
        recommenders.c.firstname.label('Recommender_Firstname')
)
stmt = stmt.select_from(members.join(recommenders, members.c.recommendedby == recommenders.c.memid))
stmt = stmt.where(members.c.memid != recommenders.c.memid)
stmt = stmt.order_by('Member_Surname', 'Member_Firstname')

In [7]:
rs = con.execute(stmt)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

       Member_Surname Member_Firstname Recommender_Surname  \
0               Bader         Florence            Stibbons   
1               Baker             Anne            Stibbons   
2               Baker          Timothy             Farrell   
3              Boothe              Tim              Rownam   
4             Butters           Gerald               Smith   
5              Coplin             Joan               Baker   
6             Crumpet            Erica               Smith   
7                Dare            Nancy            Joplette   
8             Genting          Matthew             Butters   
9                Hunt             John             Purview   
10              Jones            David            Joplette   
11              Jones          Douglas               Jones   
12           Joplette           Janice               Smith   
13          Mackenzie             Anna               Smith   
14               Owen          Charles               Smith   
15      

# Q12

In [8]:
stmt = db.select(
        facilities.c.name,
        db.func.sum(bookings.c.slots).label('member_usage')
)
stmt = stmt.select_from(facilities.join(bookings, facilities.c.facid == bookings.c.facid))
stmt = stmt.where(bookings.c.memid != 0)
stmt = stmt.group_by(facilities.c.name)
stmt = stmt.order_by(db.desc('member_usage'))

In [9]:
rs = con.execute(stmt)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

              name  member_usage
0  Badminton Court          1086
1   Tennis Court 1           957
2   Massage Room 1           884
3   Tennis Court 2           882
4    Snooker Table           860
5       Pool Table           856
6     Table Tennis           794
7     Squash Court           418
8   Massage Room 2            54


# Q13

In [10]:
stmt = db.select(
            facilities.c.name,
            db.extract("month", bookings.c.starttime).label('Month'),
            db.func.sum(bookings.c.slots).label('member_usage')
)
stmt = stmt.select_from(facilities.join(bookings, facilities.c.facid == bookings.c.facid))
stmt = stmt.where(bookings.c.memid != 0)
stmt = stmt.group_by(facilities.c.name, 'Month')

In [11]:
rs = con.execute(stmt)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
print(df)

               name  Month  member_usage
0   Badminton Court      7           165
1   Badminton Court      8           414
2   Badminton Court      9           507
3    Massage Room 1      7           166
4    Massage Room 1      8           316
5    Massage Room 1      9           402
6    Massage Room 2      7             8
7    Massage Room 2      8            18
8    Massage Room 2      9            28
9        Pool Table      7           110
10       Pool Table      8           303
11       Pool Table      9           443
12    Snooker Table      7           140
13    Snooker Table      8           316
14    Snooker Table      9           404
15     Squash Court      7            50
16     Squash Court      8           184
17     Squash Court      9           184
18     Table Tennis      7            98
19     Table Tennis      8           296
20     Table Tennis      9           400
21   Tennis Court 1      7           201
22   Tennis Court 1      8           339
23   Tennis Cour